In [1]:
# del session.logger
# del session

import multiprocessing
from train.training_session_v1 import TrainingSessionV1
from config import TrainingConfigV1
from config import SimpleConvConfig

data_partition = {
    "gwilliams2023": {
        "testing_subjects": [19, 20, 21],
        "testing_tasks": [0],
    },
    # "armeini2022": {
    #     "testing_subjects": [],
    #     "testing_tasks": [8, 9],
    # },
}

model_config = SimpleConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=80,
    hidden_dim=384,
    dropout=0.2,
    initial_batch_norm=True,
    # Sensor layout settings
    layout_dim=2,
    layout_proj=True,
    layout_scaling="minmax",
    # Merger with spatial attn
    merger=False,
    merger_emb_type=None,
    merger_emb_dim=0,
    merger_channels=0,
    merger_dropout=0.0,  # Float
    merger_conditional=None,
    # Inital
    initial_linear=384,
    initial_depth=1,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim=None,  # input or hidden_dim
    # Conv layer overall structure
    depth=6,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.2,
    batch_norm=True,
    half=True,
    cnn_pos_encoding=False,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input=None,
    transformer_encoder_emb=None,
    transformer_encoder_layers=0,
    transformer_encoder_heads=0,
    # Conformer encoder variant
    rnn_type="conformer",
    depthwise_conv_kernel_size=31,
    use_group_norm=True,
    convolution_first=False,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

config = TrainingConfigV1(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Ada lora settings
    adalora_init_r=8,
    adalora_target_r=4,
    adalora_tinit=1000,
    adalora_tfinal=180000,
    adalora_deltaT=1000,
    adalora_lora_alpha=16,
    adalora_lora_dropout=0.2,
    adalora_total_step=180000,
    # Pre-processing parameters
    # Brain
    new_freq=200,
    frequency_bands={"all": (0.5, 80)},
    max_random_shift=1.0,
    window_size=4,
    window_stride=1,
    brain_clipping=None,
    baseline_window=0.5,
    notch_filter=True,
    scaling="both",
    delay=0.15,
    # Hyperparameters
    learning_rate=3e-4,
    weight_decay=1e-4,
    epochs=50,
    batch_size=32,
    random_test_size=10,
    seed=42,
    mel_alignment_objectives={
        "clip_loss": 0.6,
        "mse_loss": 0.4,
        "commitment_loss": 0.0,
    },
    latent_alignment_objectives={
        "cosine_similarity": 0.4,
        "mse_loss": 0.4,
        "clip_loss": 0.2,
    },
    latent_alignment_layers=[-1],
)

session = TrainingSessionV1(
    config=config,
    studies={study: "audio" for study in data_partition.keys()},
    data_path="data",
    save_path="saves/phase2/objectives/CLIP_MSE_COSINE_LAST_LAYER",
    clear_cache=False,
    cache_name="cache/1",
)

try:
    session.train(
        device="cuda",
        buffer_size=30,
        num_workers=(multiprocessing.cpu_count() - 2),
        max_cache_size=400,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print("Exited")

# try:
#     session.pre_process_all_recordings(
#         buffer_size=30, num_workers=multiprocessing.cpu_count() - 2, max_cache_size=400
#     )
# except KeyboardInterrupt as e:
#     print("Exited")

Loading Gwilliams2023 with batch type audio
Data partitioned on studies ['gwilliams2023'].
Train: 135, Unseen Task: 12, Unseen Subject: 45, Unseen Both: 4.

SimpleConv initialized with 9295984 parameters, cond: ['study', 'subject']
Merger False, merger channels 0
ConvBlocks: 6, hidden_dim: 384, params 8858112


2025-01-29 02:51:54,183	INFO worker.py:1841 -- Started a local Ray instance.
Training Epoch 1:   5%|▌         | 7/135 [03:14<43:41, 20.48s/it]  W0129 02:55:19.139000 94789 site-packages/torch/_dynamo/convert_frame.py:844] [2/8] torch._dynamo hit config.cache_size_limit (8)
W0129 02:55:19.139000 94789 site-packages/torch/_dynamo/convert_frame.py:844] [2/8]    function: 'forward' (/home/ubuntu/brain-decoding/models/common.py:152)
W0129 02:55:19.139000 94789 site-packages/torch/_dynamo/convert_frame.py:844] [2/8]    last reason: 2/1: len(L['self'].position_getter._cache) == 1                  
W0129 02:55:19.139000 94789 site-packages/torch/_dynamo/convert_frame.py:844] [2/8] To log all recompilation reasons, use TORCH_LOGS="recompiles".
W0129 02:55:19.139000 94789 site-packages/torch/_dynamo/convert_frame.py:844] [2/8] To diagnose recompilation issues, see https://pytorch.org/docs/main/torch.compiler_troubleshooting.html.
Training Epoch 1:  32%|███▏      | 43/135 [14:49<33:09, 21.63s/it]

Error in processing Gwilliams2023 17 1 3.
Error in epoch 1, Gwilliams2023 17 1 3. Skipping. 'NoneType' object has no attribute 'keys'
Deleted directory: saves/phase2/objectives/CLIP_MSE_COSINE_LAST_LAYER/error_epoch_1


AttributeError: 'NoneType' object has no attribute 'keys'

In [ ]:
import torch

torch.cuda.get_device_capability()

(8, 6)

In [4]:
for name, param in session.model.encoder.named_parameters():
    if "lora_A" in name or "lora_B" in name or "lora_E" in name:
        print(name)

model.layers.0.self_attn.k_proj.lora_A.default
model.layers.0.self_attn.k_proj.lora_B.default
model.layers.0.self_attn.k_proj.lora_E.default
model.layers.0.self_attn.v_proj.lora_A.default
model.layers.0.self_attn.v_proj.lora_B.default
model.layers.0.self_attn.v_proj.lora_E.default
model.layers.0.self_attn.q_proj.lora_A.default
model.layers.0.self_attn.q_proj.lora_B.default
model.layers.0.self_attn.q_proj.lora_E.default
model.layers.0.self_attn.out_proj.lora_A.default
model.layers.0.self_attn.out_proj.lora_B.default
model.layers.0.self_attn.out_proj.lora_E.default
model.layers.0.fc1.lora_A.default
model.layers.0.fc1.lora_B.default
model.layers.0.fc1.lora_E.default
model.layers.0.fc2.lora_A.default
model.layers.0.fc2.lora_B.default
model.layers.0.fc2.lora_E.default
model.layers.1.self_attn.k_proj.lora_A.default
model.layers.1.self_attn.k_proj.lora_B.default
model.layers.1.self_attn.k_proj.lora_E.default
model.layers.1.self_attn.v_proj.lora_A.default
model.layers.1.self_attn.v_proj.lora_B